In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
import threading
import queue
#import time

In [2]:
cam = 0
fps = 240 #Maximum achivable was 163.06

model_version = "my_model_20.0.keras"
max_frames = 3  # Number of frames to evaluate

image_path = "no_bottle.jpg"
frame_buffer = []
frame_queue = queue.Queue(maxsize=20)

Im_Width = 128
Im_Height = 128

#crop
x = 120
y = 0
w = 400
h = 200

max_brightness = 175
#max_edgeRichness = 

#run = True
stop_event = threading.Event()

In [3]:
def crop_image(frame, x, y, w, h):
    return frame[y:y+h, x:x+w]

In [4]:
def measure_brightness(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
    brightness = np.mean(gray)
    return brightness 

In [5]:
def measure_edge_richness(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    edges = cv2.Canny(gray, 200, 400)  # Apply Canny edge detection
    edge_richness = np.sum(edges)  # Sum of edge pixel intensities
    return edge_richness

In [6]:
def select_best_frame(frame_buffer, x, y, w, h, max_brightness):

    best_frame = None
    highest_brightness = 0
    highest_edge_richness = 0

    for frame in frame_buffer:

        #Crop
        cropped_frame = crop_image(frame, x, y, w, h)

        # Measure brightness
        brightness = measure_brightness(cropped_frame)

        if brightness > max_brightness :
            # Update the best frame if this one is better
            if brightness > highest_brightness:
                highest_brightness = brightness
                best_frame = frame  # Save the frame as the best frame
        
        elif brightness > 150 and brightness < 176:
            # Apply edge detection
            edge_richness = measure_edge_richness(cropped_frame)
            # Update the best frame if this one is better
            if edge_richness > highest_edge_richness:
                highest_edge_richness = edge_richness
                best_frame = frame  # Save the cropped frame as the best frame


    return best_frame  # Returns None if no frames meet the threshold


In [7]:
def process_frame(frame, target_size):

    # Resize the frame to the target size
    resized_frame = cv2.resize(frame, target_size)

    # Normalize pixel values to [0, 1]
    normalized_frame = resized_frame / 255.0

    # Convert to NumPy array and add batch dimension
    frame_array = img_to_array(normalized_frame)
    frame_array = np.expand_dims(frame_array, axis=0)  # Shape: (1, height, width, channels)

    return frame_array

In [8]:
def predict_frame(frame, model, target_size):

    # Preprocess the frame
    frame_array = process_frame(frame, target_size)

   # Make a prediction
    prediction = model.predict(frame_array)[0][0]  # Get the confidence score
    print(f"Raw Prediction: {prediction}")

    # Interpret the prediction
    if prediction > 0.5:
        return "Non-Defective", prediction       
        
    else:
        return "Defective", prediction

In [9]:
def capture(cam, fps, frame_buffer, frame_queue, max_frames, stop_event):
    
    cap = cv2.VideoCapture(cam)  # Open webcam
    cap.set(cv2.CAP_PROP_FPS, fps)  # Set the desired FPS

    while not stop_event.is_set():
        if not cap.isOpened():
            print("Error: Cannot open camera.")
            return

        while not stop_event.is_set():
            ret, frame = cap.read()
            if not ret:
                print("Failed to capture frame. Exiting...")
                break

            frame_buffer.append(frame)

            # Yield the buffer when it's full
            if len(frame_buffer) == max_frames:
                frame_queue.put(frame_buffer.copy())  # Add a copy of the buffer
                frame_buffer.clear()  # Clear the buffer for the next batch

    cap.release()

In [10]:
# import cv2
# import threading
# import queue
# import time

# def capture_frames(cam, fps, frame_queue, buffer_size=30):
#     """
#     Captures frames from the camera and stores them in a queue for processing.
    
#     Args:
#         cam (int): Camera index.
#         fps (int): Frames per second.
#         frame_queue (queue.Queue): Queue to store frame buffers.
#         buffer_size (int): Number of frames per buffer.
#     """
#     cap = cv2.VideoCapture(cam)  # Open webcam
#     cap.set(cv2.CAP_PROP_FPS, fps)  # Set FPS

#     if not cap.isOpened():
#         print("Error: Cannot open camera.")
#         return

#     frame_buffer = []

#     while True:
#         ret, frame = cap.read()
#         if not ret:
#             print("Failed to capture frame. Exiting...")
#             break

#         frame_buffer.append(frame)

#         # Add the buffer to the queue when full
#         if len(frame_buffer) == buffer_size:
#             frame_queue.put(frame_buffer.copy())  # Add a copy of the buffer
#             frame_buffer.clear()  # Clear the buffer for the next batch

#     cap.release()

# def process_frames(frame_queue):
#     """
#     Processes frame buffers retrieved from the queue.
    
#     Args:
#         frame_queue (queue.Queue): Queue to retrieve frame buffers from.
#     """
#     while True:
#         try:
#             # Get a buffer from the queue
#             frame_buffer = frame_queue.get(timeout=5)  # Wait for buffer (5 sec timeout)
#             print(f"Processing buffer with {len(frame_buffer)} frames")
            
#             # Simulate processing time
#             time.sleep(1)

#         except queue.Empty:
#             print("No buffers left to process. Exiting...")
#             break

# if __name__ == "__main__":
#     cam = 0  # Default camera index
#     fps = 30  # Desired FPS
#     buffer_size = 30  # Number of frames per buffer
#     frame_queue = queue.Queue(maxsize=10)  # Queue with a maximum of 10 buffers

#     # Create threads
#     capture_thread = threading.Thread(target=capture_frames, args=(cam, fps, frame_queue, buffer_size))
#     processing_thread = threading.Thread(target=process_frames, args=(frame_queue,))

#     # Start threads
#     capture_thread.start()
#     processing_thread.start()

#     # Wait for threads to complete
#     capture_thread.join()
#     processing_thread.join()

#     print("All tasks completed.")


In [11]:
def runModel(model_version, frame_queue, x, y, w, h, max_brightness, stop_event, target_size=(Im_Width, Im_Height)):

    # Load the model
    print("Loading model...") 
    model = load_model(model_version)
    print("Model loaded successfully.")
    
    while True:
        frame_buffer = frame_queue.get()
        best_frame = select_best_frame(frame_buffer, x, y, w, h, max_brightness)
        # Process and predict
        if best_frame is not None:
            feedback, confidence = predict_frame(best_frame, model, target_size)
            print(f"Prediction: {feedback} (Confidence: {confidence:.2f})")

            prob = abs(round(confidence,2)- 0.5)*200

            # Display feedback on the frame
            cv2.putText(best_frame, f"Prediction: {feedback}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(best_frame, f"Confidence: {prob:.0f}", (10, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            # Display the best frame
            cv2.imshow("Best Frame", best_frame)

        else:
            # Load and display the "no-bottle" image in the same window
            image = cv2.imread(image_path)
            cv2.putText(image, "No Bottle!", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.imshow("Best Frame", image)

        # Exit on 'ESC' key
        if cv2.waitKey(1) & 0xFF == 27:
            #run = False
            stop_event.set()
            break

    cv2.destroyAllWindows()
    #return run

In [12]:
capture_thread = threading.Thread(target=capture, args=(cam, fps, frame_buffer, frame_queue, max_frames, stop_event))
runModel_thread = threading.Thread(target=runModel, args=(model_version, frame_queue, x, y, w, h, max_brightness, stop_event))

In [ ]:
if __name__ == "__main__":

    # Start the threads
    capture_thread.start()
    runModel_thread.start()

    # Wait for threads to complete
    capture_thread.join()
    #frame_queue.join()
    runModel_thread.join()


Loading model...
Model loaded successfully.
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 822ms/step
Raw Prediction: 0.0012344869319349527
Prediction: Defective (Confidence: 0.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Raw Prediction: 0.0016159956576302648
Prediction: Defective (Confidence: 0.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Raw Prediction: 0.0014480955433100462
Prediction: Defective (Confidence: 0.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Raw Prediction: 0.0019390281522646546
Prediction: Defective (Confidence: 0.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Raw Prediction: 0.0021176955197006464
Prediction: Defective (Confidence: 0.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Raw Prediction: 0.0020083533599972725
Prediction: Defective (Confidence: 0.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Raw Prediction: 0.0019190008752048016
Prediction: Defective (Confidence: 0.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Raw Prediction: 0.0016719818813726306
Prediction: Defective (Confidence: 0.00)
1/1 ━━━━━━━━━━━━━━━